<a href="https://colab.research.google.com/github/gchecon/ArquivosDiversos/blob/main/ExtrairPubPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de PDF diretamente do site "https://www.tce.sp.gov.br/publicacoes"

In [2]:
import requests # Utilização desta biblioteca simples
import sys      # Para finalização em caso de erro no programa compilado
import re       # Manipulação de String
import os.path  # Verificação se um arquivo existe para não baixar

In [3]:
url = "https://www.tce.sp.gov.br/publicacoes?tipo=All&busca=&page="
url_base = "https://www.tce.sp.gov.br/publicacoes"
pag = 0
url_work = url+str(pag)
r = requests.get(url_work)
lista = []
idx = 16
while idx > 1:  
    # selecionar todas referências iniciadas por "https://www.tce.sp.gov.br/publicacoes"
    # neste momento, para fins de teste, pegar apenas a primeira página
    pag += 1
    idx = len(re.findall(url_base, r.text))
    str1 = r.text
    i = 0
    while i < idx:
        i += 1
        m = re.search(url_base, str1)
        str1 = str1[m.start():len(str1)]
        str2 = str1[37]
        if str2 == '/':
            str2 = str1[0:37]
            str1 = str1[37:len(str1)]
            str2 = str2 + str1[0:re.search('"',str1).start()]
            lista.append(str2)
        else:
            str1 = str1[37:len(str1)]   
    url_work = url+str(pag)
    r = requests.get(url_work)
    # print(lista)

# Baixar e Salvar PDFs em memória

In [4]:
# lista[] contém todas as urls que apontam para os artigos. 
# O programa agora deve criar outra lista com os apontamentos para os arquivos PDF
# Na url, existem várias citações do mesmo arquivo. Desta forma, somente a primeira ocorrência é utilizada
lista_pdf = []
total_de_arquivos = 0
for str1 in lista:
    r = requests.get(str1)
    if r.status_code == 200:
        # Baixar e salvar PDFs
        str2 = r.text
        pos = re.search('.pdf', str2).start()
        lpos = 4
        while pos > 0:
            pos = pos - 1
            lpos += 1
            if str2[pos:pos+5] == 'https': 
                lista_pdf.append(str2[pos:pos+lpos])
                total_de_arquivos += 1
                break
            else: 
                continue
    else:
        print("Não foi possível acessar o link ",str1)
print("Total de arquivos: %i" % total_de_arquivos)

Total de arquivos: 131


# Salvar todos os links que apontam para os PDFs, juntamente com os arquivos

In [5]:
# Preparação do aquivo que conterá todos as urls que apontam para os arquivos para
# posterior montagem de um iframe http
str_end = './PubResumo/Referencia/Enderecos.txt'
try: 
    end_list = open(str_end, 'w')
except:
    print("Problemas na abertura do arquivo "+str_end)

# Salvar todos os arquivos da list_pdf em diretório específico

In [6]:
# Salvar todos os arquivos da list_pdf em diretório específico
grav = 0
existentes = 0
falha = 0
log = []
for str1 in lista_pdf:
    # Extrair nome do arquivo
    pos = len(str1)
    while pos > 0:
        pos = pos - 1
        if str1[pos] == '/':
            break
    str2 = str1[pos+1:len(str1)]
    str2 = './PubResumo/PDF/'+str2
    if os.path.exists(str2) == True:
        log.append("Existente: "+str2)
        end_list.write(str1+'\n')
        existentes += 1
    else:
        r = requests.get(str1)
        if r.status_code == requests.codes.OK:
            with open(str2, 'wb') as arquivo:
                arquivo.write(r.content)
                log.append("Gravado:   "+str2)
                end_list.write(str1+'\n')
                grav += 1
                arquivo.close()
        else:
            falha += 1
            log.append("Erro:      "+str2)
log.append("Total de Gravados:.. "+str(grav))
log.append("Total de Existentes: "+str(existentes))
log.append("Total de Falhas:.... "+str(falha))
with open("Relatorio.txt","w") as rel_log:
    for str3 in log:
        rel_log.write(str3+'\n')
end_list.close()
rel_log.close()